In [5]:
import os

In [2]:
%pwd

'/home/arunavo/End_to_end_with_MLFlow_and_DVC/research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'/home/arunavo/End_to_end_with_MLFlow_and_DVC'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [27]:
CONFIG_FILE_PATH

PosixPath('config/config.yaml')

In [10]:
read_yaml(CONFIG_FILE_PATH)

[2024-08-24 13:46:53,074: INFO: common: yaml file: (path_to _yaml) loaded successfully]


ConfigBox({'artifacts root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1y3Y8i9J3Gj9t72ZzxtXOlT81Mn6_QDeZ/view?usp=drive_link', 'local_data_files': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}})

In [15]:
class ConfigurationManager:
    def __init__(
    self,
    config_filepath = CONFIG_FILE_PATH,
    params_filepath =  PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file = config.local_data_files,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config
    
    

In [16]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> str:
        """
          Fetch data from url
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            False

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """ 

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-24 13:49:08,228: INFO: common: yaml file: (path_to _yaml) loaded successfully]
[2024-08-24 13:49:08,234: INFO: common: yaml file: (path_to _yaml) loaded successfully]
[2024-08-24 13:49:08,238: INFO: common: created directory at: artifacts]
[2024-08-24 13:49:08,242: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-24 13:49:08,250: INFO: 3261427307: Downloading data from https://drive.google.com/file/d/1y3Y8i9J3Gj9t72ZzxtXOlT81Mn6_QDeZ/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1y3Y8i9J3Gj9t72ZzxtXOlT81Mn6_QDeZ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1y3Y8i9J3Gj9t72ZzxtXOlT81Mn6_QDeZ&confirm=t&uuid=2d87a751-9339-4a99-b7fb-166158c33271
To: /home/arunavo/End_to_end_with_MLFlow_and_DVC/artifacts/data_ingestion/data.zip
100%|██████████| 124M/124M [00:09<00:00, 13.2MB/s] 

[2024-08-24 13:49:22,853: INFO: 3261427307: Download data from https://drive.google.com/file/d/1y3Y8i9J3Gj9t72ZzxtXOlT81Mn6_QDeZ/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
